In [113]:
import requests
import argparse
import time
import json
import StringIO
import gzip
import csv
import codecs
import pandas as pd
import re

from bs4 import BeautifulSoup

In [114]:
df_amazon = {}
df_apple = {}
df_google = {}
df_facebook = {}
df_uber = {}

target_size = 250
count_amz= 0
count_app= 0
count_ggl= 0
count_fb= 0
count_ub= 0


domainlist = ["https://techcrunch.com/2019/*", "https://techcrunch.com/2018/*"]

# list of available indices
index = "2019-13"

#
# Searches the Common Crawl Index for a domain.
#
def search_domain(domainlist):

    record_list = []
    
    
    
    for domain in domainlist:
        
        print "[*] Trying target domain: %s" % domain
        
        
        cc_url  = "http://index.commoncrawl.org/CC-MAIN-%s-index?" % index
        cc_url += "url=%s&output=json" % domain
        
#         cc_url  = "https://index.commoncrawl.org/CC-MAIN-2019-13-index?url=https%3A%2F%2Ftechcrunch.com%2F2019%2F*&output=json"
        
        response = requests.get(cc_url)
        
        if response.status_code == 200:
            
            records = response.content.splitlines()
            
            for record in records:
                record_list.append(json.loads(record))
            
            print "[*] Added %d results." % len(records)
            
    
    print "[*] Found a total of %d hits." % len(record_list)
    
    return record_list        

#
# Downloads a page from Common Crawl - adapted graciously from @Smerity - thanks man!
# https://gist.github.com/Smerity/56bc6f21a8adec920ebf
#
def download_page(record):

    offset, length = int(record['offset']), int(record['length'])
    offset_end = offset + length - 1

    # We'll get the file via HTTPS so we don't need to worry about S3 credentials
    # Getting the file on S3 is equivalent however - you can request a Range
    prefix = 'https://commoncrawl.s3.amazonaws.com/'
    
    # We can then use the Range header to ask for just this set of bytes
    resp = requests.get(prefix + record['filename'], headers={'Range': 'bytes={}-{}'.format(offset, offset_end)})
    
    # The page is stored compressed (gzip) to save space
    # We can extract it using the GZIP library
    raw_data = StringIO.StringIO(resp.content)
    f = gzip.GzipFile(fileobj=raw_data)
    
    # What we have now is just the WARC response, formatted:
    data = f.read()
    
    response = ""
    
    if len(data):
        try:
            warc, header, response = data.strip().split('\r\n\r\n', 2)
        except:
            pass
            
    return response


def extract_content(html_content,url):
    
    global count_amz,count_app, count_ggl, count_fb,count_ub
    
    url = url.rsplit('?',1)[0]
    
    if not url.startswith('https'):
        return
    
    try:

        parser = BeautifulSoup(html_content)

        # kill all script and style elements
#         for script in parser(["script", "style"]):
#             script.decompose()


        page_body = parser.find_all("div", class_="article-content")[0].get_text()
        page_body = re.sub('\s+', ' ', page_body).strip().lower()
        
        if 'moved permanently' in page_body:
            return

        if "amazon" in page_body:
            df_amazon[url]= page_body
            count_amz += 1
#             print "amazon::"+ str(count_amz) + ":" +url
            
        if "apple" in page_body:
            df_apple[url]= page_body
            count_app += 1
#             print "apple::"+ str(count_app) + ":" +url
            
        if "google" in page_body:
            df_google[url]= page_body
            count_ggl += 1
#             print "google::"+ str(count_ggl) + ":" +url
            
        if "facebook" in page_body:
            df_facebook[url]= page_body
            count_fb += 1
#             print "facebook::"+ str(count_fb) + ":" +url
            
        if "uber" in page_body:
            df_uber[url]= page_body
            count_ub += 1
#             print "uber::"+ str(count_ub) + ":" +url


        if count_ggl%30 ==0:
            print "amazon::"+ str(count_amz) + ":" +"apple::"+ str(count_app) + ":" +"google::"+ str(count_ggl) + ":" +"facebook::"+ str(count_fb) + ":" +"uber::"+ str(count_ub)
            
    except:
        pass
    
    



record_list = search_domain(domainlist)

for record in record_list:
    
    html_content = download_page(record)
    
#     print "[*] Retrieved %d bytes for %s" % (len(html_content),record['url'])
    
    if len(html_content)>0:
        
        if count_amz <target_size or count_app <target_size  or count_ggl <target_size  or count_fb <target_size  or count_ub <target_size:
            extract_content(html_content,record['url'])
        else:
            break
            
print '!!ENDD!!!!'



[*] Trying target domain: https://techcrunch.com/2019/*
[*] Added 3366 results.
[*] Trying target domain: https://techcrunch.com/2018/*
[*] Added 10607 results.
[*] Found a total of 13973 hits.
amazon::0:apple::0:google::0:facebook::0:uber::1
amazon::0:apple::0:google::0:facebook::0:uber::2
amazon::0:apple::0:google::0:facebook::0:uber::3
amazon::0:apple::1:google::0:facebook::0:uber::3
amazon::0:apple::1:google::0:facebook::0:uber::3
amazon::20:apple::25:google::30:facebook::13:uber::8
amazon::20:apple::25:google::30:facebook::13:uber::8
amazon::20:apple::25:google::30:facebook::13:uber::8
amazon::20:apple::25:google::30:facebook::13:uber::8
amazon::20:apple::25:google::30:facebook::13:uber::8
amazon::20:apple::25:google::30:facebook::13:uber::8
amazon::20:apple::25:google::30:facebook::13:uber::8
amazon::45:apple::40:google::60:facebook::26:uber::12
amazon::45:apple::40:google::60:facebook::26:uber::12
amazon::45:apple::40:google::60:facebook::26:uber::12
amazon::45:apple::40:google:

In [123]:
df_amz = pd.DataFrame.from_dict(df_amazon, orient='index')
df_app = pd.DataFrame.from_dict(df_apple, orient='index')
df_ggl = pd.DataFrame.from_dict(df_google, orient='index')
df_fb = pd.DataFrame.from_dict(df_facebook, orient='index')
df_ub = pd.DataFrame.from_dict(df_uber, orient='index')

In [124]:
df_amz = df_amz.sample(n = 120)
df_app = df_app.sample(n = 120)
df_ggl = df_ggl.sample(n = 120)
df_fb = df_fb.sample(n = 120)
df_ub = df_ub.sample(n = 120)

In [129]:
df_amz.to_csv('data_files/cc/amazon.csv', encoding='utf-8', mode='a', header=False, index = False)
df_app.to_csv('data_files/cc/apple.csv', encoding='utf-8', mode='a', header=False, index = False)
df_ggl.to_csv('data_files/cc/google.csv', encoding='utf-8', mode='a', header=False, index = False)
df_fb.to_csv('data_files/cc/facebook.csv', encoding='utf-8', mode='a', header=False, index = False)
df_ub.to_csv('data_files/cc/uber.csv', encoding='utf-8', mode='a', header=False, index = False)